<a href="https://colab.research.google.com/github/RaphaPUC/Entrega-MVP---Machine-Learning/blob/main/PUC_MVP_Projeto_de_Machine_Learning_Classificacao(Analise_de_Sentimento).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP PUC-RIO

# Descrição do Problema

O problema é a dificuldade dos usuários em encontrar filmes e séries que correspondam aos seus gostos individuais nos serviços de streaming, devido à vasta quantidade de opções disponíveis.

A hipótese é que os usuários tendem a preferir conteúdo que reflete sentimentos positivos em resenhas e avaliações anteriores.

Os dados selecionados devem incluir resenhas de filmes com sentimentos claramente identificáveis e avaliações dos usuários.

 O dataset consiste em resenhas de filmes do IMDb, incluindo o texto da resenha, a avaliação associada (positiva ou negativa), e o ID do filme.


# Preparação de Dados
O dataset já está separado em conjuntos de treino e teste, conforme indicado pelo código data.load_data() que retorna (x_train, y_train), (x_test, y_test). Não há menção a um conjunto de validação separado, mas é possível criar um usando uma fração do conjunto de treino, como mostrado no método model.fit() onde validation_split=0.2 reserva 20% dos dados de treino para validação.
Operações de transformação de dados incluirão a normalização do texto e vetorização das palavras.
A validação cruzada é útil para avaliar modelos, mas pode ser demorada e não é sempre necessária.
No caso de Análise de Sentimento, a validação cruzada pode não ser necessária por conta do conjunto de dados do IMDB ser grande, então temos muitos dados para treinar. E o modelo já usa uma parte dos dados de treino para validação durante o treinamento. Então a validação cruzada seria demorada devido ao tamanho do conjunto de dados.
A seleção de atributos se concentrará em palavras-chave e sentimentos expressos nas resenhas.


# Modelagem e Treinamento
Foi escolhido o Keras, um modelo popular que transforma palavras em números e entende a relação entre elas.
 Foram ajustados alguns detalhes do modelo para garantir que ele aprenda da melhor maneira.
 O modelo foi treinado corretamente, sem problemas de underfitting, o que significa que ele aprendeu bem com os dados.
 É possível melhorar ainda mais o modelo ajustando alguns detalhes, como a taxa de aprendizado.


# Avaliação de Resultados
 Métricas de Avaliação: Foi executada uma métrica chamada SparseCategoricalAccuracy para avaliar o modelo.

O modelo foi treinado com a base de treino e testado com a base de teste, conforme indicado pelas etapas de ajuste (fit) e avaliação (evaluate).

 Foi implementado o modelo LSTM para análise de sentimento. Mas que é adequado para sequências de texto devido à sua capacidade de capturar dependências de longo prazo.

# Análise de Sentimento - base IMDB

Etapas:

- Carregar dados
- Definir modelo Keras
- Compilar modelo Keras
- Ajustar (fit) modelo Keras
- Avaliar (evalute) modelo Keras
- Faça previsões (predict)

## Carregar dados

In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#Para evitar que o console fique poluído com mensagens desnecessárias.

In [ ]:
#Importando bibliotecas
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
data = keras.datasets.imdb

(x_train, y_train), (x_test, y_test) = data.load_data()
#Importando dataset IMB, onde x_train é a lista de críticas de filmes e y_train de rótulos de sentimento (1 = positivo, 0 = negativo). x_test e y_test são os dados de teste de mesma estrutura.

In [ ]:
x_train.shape, y_train.shape
#x_train é um array de listas, onde cada lista representa uma crítica de filme. y_train é um array de rótulos, onde cada rótulo é 0 (para uma crítica negativa) ou 1 (para uma crítica positiva).

In [ ]:
x_test.shape, y_test.shape
#x_test.shape retorna o número total de críticas de filmes no conjunto de teste. y_test.shape retornará o número total de rótulos no conjunto de teste.

In [ ]:
x_train[0]
#É a primeira crítica de filme no conjunto de treinamento. Cada crítica é uma lista de índices de palavras. Cada índice corresponde a uma palavra específica no vocabulário.

In [ ]:
len(x_train[0])
#Retorna o comprimento da primeira crítica de filme no conjunto de treinamento, ou seja, o número de palavras na primeira crítica.

In [ ]:
y_train[:5]
#Retorna os primeiros 5 rótulos no conjunto de treinamento. No contexto do conjunto de dados IMDB, cada rótulo é 0 (para uma crítica negativa) ou 1 (para uma crítica positiva).
#Portanto, y_train[:5] retornará os rótulos de sentimento das primeiras 5 críticas no conjunto de treinamento.

In [ ]:
np.unique(y_train, return_counts=True)
#Retorna os valores únicos em y_train e conta quantas vezes cada um aparece.
#A expressão vai contar quantas críticas negativas e positivas existem no conjunto de treinamento.

In [ ]:
word_index = data.get_word_index()
# word_index
#Criando um dicionário de palavras para índices a partir do conjunto de dados IMDB.

In [ ]:
len(word_index)
#Retorna o número de entradas no dicionário.

In [ ]:
word_index["the"]
#Retorna o índice único que representa a palavra “the”.

In [ ]:
for chave, valor in word_index.items():
    if valor == 1:
        print(chave)
        #Retorna o índice único que representa o valor “1”.

In [ ]:
for review in x_train[:5]:
    print(len(review))
    #Imprime o número de palavras nas primeiras 5 críticas no conjunto de treinamento.

# **Conceito de Token**

No processamento de linguagem natural, como na análise de dados de texto do IMDB, um “token” é basicamente uma palavra. “Tokenização” é o processo de dividir o texto em palavras individuais, que chamamos de “tokens”. Isso é importante porque ajuda a transformar o texto em uma forma que os modelos de aprendizado de máquina podem entender e usar. Por exemplo, uma frase é dividida em palavras individuais, e cada palavra é um “token”.

Considere a frase "Aprendendo processamento de linguagem natural". Na tokenização, esta frase seria dividida em tokens individuais. Cada palavra representa um token:

- Token 1: "Aprendendo"
- Token 2: "processamento"
- Token 3: "de"
- Token 4: "linguagem"
- Token 5: "natural"

Neste exemplo, a frase original é decomposta em palavras isoladas, cada uma considerada um token.

Um exemplo onde um token não é necessariamente uma palavra pode ser encontrado na tokenização baseada em caracteres ou sílabas. Por exemplo, na frase "Incrível", a tokenização por sílabas resultaria nos tokens "In", "crí", "vel". Aqui, cada sílaba é tratada como um token distinto, ao invés de cada palavra inteira. Este tipo de tokenização pode ser útil em tarefas de processamento de linguagem natural que exigem uma análise mais detalhada do texto, como na compreensão de padrões fonéticos ou na análise de idiomas com estruturas de palavras complexas.

- PAD é o token usado para preenchimento. Nós preenchemos todas as sequências para o mesmo comprimento, que é o comprimento da sequência mais longa.
- START é o token usado para marcar o início de uma sequência.
- UNK é o token usado para marcar palavras desconhecidas (palavras que não estão no vocabulário).
- UNUSED é o token usado para preencher as posições não utilizadas em uma sequência.

Preparando o dicionário para ser usado em NLP, onde cada palavra é mapeada para um valor único, e quatro tokens especiais são adicionados para ajudar no processamento das sequências.

In [ ]:
word_index = {chave: (valor + 3) for chave, valor in word_index.items()}
#O código está atualizando o dicionário word_index ao adicionar 3 a cada valor (índice) no dicionário.

In [ ]:
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3
#Adicionando quatro novas palavras ao word_index. Essas palavras são especiais e usadas para tarefas específicas ao lidar com texto:
#"<PAD>": Usado para preencher o texto para que todas as críticas tenham o mesmo tamanho.
#"<START>": Marca o início de uma crítica.
#"<UNK>": Representa uma palavra desconhecida ou muito rara.
#"<UNUSED>"": Um espaço reservado para palavras que não são usadas.

In [ ]:
for chave, valor in word_index.items():
    if valor == 1:
        print(chave)
        #Imprimindo a chave (palavra) cujo valor (índice) é 1.

In [ ]:
def decode_review(text, index):
    reverse_index = {value: key for key, value in index.items()}
    return " ".join([reverse_index.get(value, "<UNK>") for value in text])
    #A função decode_review transforma uma lista de números em uma frase.
    #Ela faz isso usando o dicionário que mapeia números para palavras. Se um número não está no dicionário, ela usa a palavra "<UNK>".
    #Então, ela junta todas as palavras com espaços para formar uma frase.

In [ ]:
decode_review(x_train[0], word_index)
#Transforma essa lista de números de volta em uma frase legível, onde cada número é substituído pela palavra correspondente.

In [ ]:
import textwrap


def print_review(text, width=50):
    wrapper = textwrap.TextWrapper(width=width)
    print(wrapper.fill(text))
    #Permite que o código use a funcionalidade de formatação de texto do Python.
    #Isso formata o texto para que cada linha tenha no máximo a largura especificada.

In [ ]:
print_review(decode_review(x_train[0], word_index))
#Após converter os tokens na revisão de volta para palavras.
#Imprime a revisão decodificada onde cada linha tenha 50 caracteres.

https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences

In [ ]:
print_review(decode_review(x_train[3], word_index))
#decodifica a quarta revisão no conjunto de treinamento (x_train[3])

In [ ]:
x_train = keras.utils.pad_sequences(
    x_train,
    value=word_index["<PAD>"],
    padding="post",
    truncating="post",
    maxlen=256
)

x_test = keras.utils.pad_sequences(
    x_test,
    value=word_index["<PAD>"],
    padding="post",
    truncating="post",
    maxlen=256
)
#pad_sequences= Cada sequência é uma revisão de filme representada como uma lista de tokens.
#value=word_index["<PAD>"]: Isso define o valor que será usado para preencher as sequências que são mais curtas que o comprimento máximo. No caso, <PAD> é um token especial que representa o preenchimento.
#truncating="post": Isso significa que as sequências que são mais longas que o comprimento máximo serão truncadas no final.
#maxlen=256:Define o comprimento máximo para as sequências. Qualquer sequência que seja mais longa que isso será truncada e qualquer sequência que seja mais curta que isso será preenchida.

In [ ]:
print_review(decode_review(x_train[0], word_index))
#Imprimindo a revisão decodificada.
#<START>:Token especial que indica o início de uma revisão.
#<PAD>:Token especial usado para preencher todas as sequências para o mesmo comprimento.Preenchemos todas as sequências para terem o mesmo comprimento da sequência mais longa.

In [ ]:
x_train[0]
#Cada número na matriz corresponde a uma palavra específica. Por exemplo, o número 1 representa a palavra <START>.
#A sequência de números representa a ordem das palavras na revisão. Por exemplo, a sequência [1, 14, 22, 16, 43] representa a sequência de palavras <START> this film was just.
#O número 0 representa o token especial <PAD>, que é usado para preencher todas as sequências para terem o mesmo comprimento.

In [ ]:
print_review(decode_review(x_train[3], word_index))
#Imprimindo a quarta revisão (índice 3) no conjunto de treinamento, que foi decodificada de números para palavras.

In [ ]:
for review in x_train[:5]:
    print(len(review))
    #Este é um loop for que itera sobre as primeiras 5 revisões no conjunto de treinamento.
    #Imprime o comprimento da revisão.

In [ ]:
x_train.shape
#Formato dos dados de treinamento.

## Definir modelo Keras


### Embedding

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

O Embedding no Keras funciona como um tradutor que transforma números, cada um representando uma palavra, em vetores de números reais em um espaço multidimensional. Cada vetor representa uma palavra de maneira única, capturando nuances de seu significado e relação com outras palavras. Por exemplo, na representação vetorial, palavras com significados semelhantes, como "feliz" e "alegre", ficarão próximas no espaço vetorial. Esses vetores são aprendidos durante o treinamento do modelo, permitindo que a máquina compreenda e processe textos de maneira mais eficaz. Isso é fundamental em tarefas como análise de sentimentos ou tradução automática, onde a compreensão do contexto e das nuances das palavras é crucial.


### RNNS

RNNs
Uma Rede Neural Recorrente (RNN) é um tipo de rede neural projetada para processar sequências, como dados de séries temporais ou texto. RNNs são únicas porque mantêm uma memória interna de entradas anteriores, permitindo-lhes capturar informações sobre a história da sequência. No entanto, RNNs padrão frequentemente têm dificuldade em aprender dependências de longo alcance devido ao problema do gradiente desaparecendo.

RNN-rolled.png

LSTM, ou Memória de Longo e Curto Prazo, é um tipo especial de RNN que aborda essa questão. Ela inclui mecanismos chamados portões que regulam o fluxo de informações, tornando-a capaz de lembrar e utilizar informações ao longo de sequências muito mais longas. Isso torna as LSTMs mais eficazes para tarefas onde entender o contexto estendido é crucial.

In [ ]:
model = keras.Sequential()
#Inicializa um novo modelo sequencial.
model.add(keras.layers.InputLayer(input_shape=(256,)))
#Adiciona uma camada de entrada. Cada entrada para o modelo é um vetor de 256 dimensões.
model.add(keras.layers.Embedding(len(word_index), 64))
#Adiciona uma camada de incorporação. A camada de incorporação transforma inteiros positivos (índices) em vetores densos de tamanho fixo.
model.add(keras.layers.LSTM(64, dropout=0.5))
#Adiciona uma camada LSTM (Long Short-Term Memory). 64 é o número de unidades na camada LSTM. dropout=0.5, é a fração das unidades a serem descartadas durante o treinamento.
model.add(keras.layers.Dense(2, activation="sigmoid"))
#Adiciona uma camada densa (também conhecida como camada totalmente conectada). A camada densa tem 2 unidades e usa a função de ativação sigmoid.
model.summary()
#Imprime um resumo do modelo, incluindo o número total de parâmetros e a forma de saída de cada camada.



## Compilar modelo Keras

In [ ]:
optimizer = keras.optimizers.Adam(1E-4)
#Define o otimizador que será usado para treinar o modelo.
loss = keras.losses.SparseCategoricalCrossentropy()
#Define a função de perda que o modelo tentará minimizar durante o treinamento.
metric = keras.metrics.SparseCategoricalAccuracy()
#Define a métrica que será usada para avaliar o desempenho do modelo durante o treinamento.

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
#Compila o modelo com o otimizador, função de perda e métricas especificados.

## Fit modelo Keras

In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=512,
    validation_split=0.2,
    verbose=1,
)
#Treinando o modelo usando os dados de treinamento.

## Evaluate modelo Keras

In [ ]:
model.evaluate(x_test, y_test)
#Avalia o desempenho do modelo

## Predict modelo Keras

In [ ]:
plt.plot(history.history["loss"], label="loss")
#Plota a perda do modelo nos dados de treinamento em cada época.
plt.plot(history.history["val_loss"], label="val_loss")
#Plota a perda do modelo nos dados de validação em cada época.
plt.legend()
plt.show()

O gráfico ilustra a convergência e a possível ocorrência de overfitting, onde a perda de validação começa a aumentar enquanto a perda continua diminuindo. Isso sugere que o modelo pode estar se ajustando demais aos dados de treinamento e perdendo a capacidade de generalizar para dados não vistos.

In [ ]:
plt.plot(history.history["sparse_categorical_accuracy"], label="accuracy")
#Plota a precisão do modelo nos dados de treinamento em cada época.
plt.plot(history.history["val_sparse_categorical_accuracy"], label="val_accuracy")
#Plota a precisão do modelo nos dados de validação em cada época.
plt.legend()
plt.show()

É interessante porque visualiza a melhoria do desempenho ao longo do tempo, indicando como o modelo aprende e se ajusta aos dados durante o treinamento. Normalmente, espera-se que a precisão de validação acompanhe a precisão do treinamento, mas se a precisão de validação começar a diminuir enquanto a precisão do treinamento continuar aumentando, isso pode indicar overfitting.

In [ ]:
for i in range(10):
    #Loop "for" que itera sobre as primeiras 10 revisões no conjunto de teste.
    print("Label: ", y_test[i])
    #Imprime o rótulo verdadeiro da i-ésima revisão no conjunto de teste. O rótulo é o sentimento associado à revisão (positivo ou negativo).
    print("Prediction (sigmoid): ", model.predict(np.expand_dims(x_test[i], axis=0), verbose=0).flatten())
    #Faz uma previsão para a i-ésima revisão no conjunto de teste.
    print("Prediction: ", np.argmax(model.predict(np.expand_dims(x_test[i], axis=0), verbose=0)))
    #Imprime a classe prevista pelo modelo para a i-ésima revisão.
    print()

In [ ]:
print_review(decode_review(x_test[5], word_index), width=80)
#Esta função está decodificando a sexta revisão (índice 5) no conjunto de teste. As revisões são codificadas como sequências de números, onde cada número representa uma palavra específica.

In [ ]:
review_good = "The movie was great! The animation and the graphics were out of this world. I would recommend this movie."
review_bad = "The movie was terrible. The animation was poor and the graphics were awful. I would not recommend this movie."
#Criadas duas variaveis de avaliações (boa e ruim).

In [ ]:
print_review(review_good)

In [ ]:
print_review(review_bad)

In [ ]:
def encode_review(text, index):
    import string
    text = text.translate(str.maketrans("", "", string.punctuation)).lower()
    #Remove toda a pontuação do texto e converte todas as letras para minúsculas.
    return [index.get(word, 2) for word in text.split(" ")]
    #Divide o texto em palavras usando espaços como delimitadores e converte cada palavra em um número usando o índice fornecido. Se uma palavra não está no índice, ela é convertida para o número 2.

In [ ]:
encode_review(review_good, word_index)

In [ ]:
decode_review(encode_review(review_good, word_index), word_index)
#Primeiro codifica a revisão positiva (review_good) em uma sequência de números usando a função encode_review, e depois decodifica essa sequência de volta em texto usando a função decode_review.

In [ ]:
review_good = keras.utils.pad_sequences(
    #Esta função transforma uma lista de sequências (listas de números inteiros) em uma matriz 2D.
    [encode_review(review_good, word_index)],
    #Cada revisão é uma lista de números, onde cada número representa uma palavra específica.
    value=word_index["<PAD>"],
    #valor usado para preencher as sequências. Usando o número que representa a palavra especial <PAD>.
    padding="post",
    #Caso uma sequência for mais curta que o comprimento limite, os valores de preenchimento serão adicionados no final da sequência.
    truncating="post",
    #Se uma sequência for mais longa que o comprimento limite, os valores extras no final da sequência serão removidos.
    maxlen=256
    #Comprimento das sequências, todas as sequências terão este comprimento após o preenchimento/truncamento.
)

review_bad = keras.utils.pad_sequences(
    [encode_review(review_bad, word_index)],
    value=word_index["<PAD>"],
    padding="post",
    truncating="post",
    maxlen=256
)

In [ ]:
review_good

In [ ]:
review_bad

In [ ]:
model.predict(review_good)

In [ ]:
model.predict(review_bad)